In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/clase_11/clase_11/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Pandas 2 - Pivot table

<a id="section_toc"></a> 
## Tabla de Contenidos

[Intro](#section_intro)

[Dataset](#section_dataset)

[pivot_table](#section_pivot)

[pivot_table multi level](#section_pivot_multi_level)


---


<a id="section_intro"></a> 
## Intro

[volver a TOC](#section_toc)


Una tabla pivote o tabal dinámica es una herramienta de resumen que está disponible generalmente en programas de hojas de cálculo.

Crea medidas de reumne por una o más keys, usando esas claves como etiquetas de filas o columnas.

Pandas provee un método `pivot_table` sobre DataFrame y también como una función de pandas

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html


<a id="section_dataset"></a> 
## Dataset

[volver a TOC](#section_toc)

En esta práctica guiada usaremos los datos que provee seaborn en el dataset "titanic"

Este dataset también puede descargarse aquí https://github.com/mwaskom/seaborn-data/blob/master/titanic.csv

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
data = sns.load_dataset('titanic')

Miremos el tamaño y los primeros registros del dataset

In [3]:
data.shape

(891, 15)

In [4]:
data.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


<a id="section_pivot"></a> 
## `pivot_table`

[volver a TOC](#section_toc)


Analizamos la supervivencia por sexo y clase.

El valor de argumento aggfunc por default es mean para el método `DataFrame.pivot_table`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html

In [5]:
data.pivot_table('survived', index='sex', columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


Los argumentos del método pivot_table son
* el campo sobre el que vamos a calcular la función de agregación, en este caso `mean`
* index define la columna del DataFrame original cuyos valores sirven de índice de la tabla dinámica
* columns define la columna del DataFrame original cuyos valores sirven de columnas de la tabla dinámica

En esta tabla vemos que sobrevivieron más mujeres que hombre, y a clase más alta mayor proporción de sobrevivientes.

<a id="section_pivot_multi_level"></a> 
## `pivot_table` multi level

[volver a TOC](#section_toc)


Analicemos grupos de edad como tercera dimensión.
 
Para eso generamos categorías dependiendo de los valores de la variable 'age' usando `cut`

La función `cut` divide los valores de una variable en intervalos discretos, con los límites especificados en el argumento `bins`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html

In [6]:
age_categories = pd.cut(data.age, [0, 18, 80])
data.pivot_table('survived', ['sex', age_categories], 'class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

Agreguemos información de la tarifa definiendo categorias de la variable `fare` de acuerdo al cuantilo al que pertenece cada valor.

Usamos la función `qcut` para calcular estas categorías

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html

Definimos dos categorías de `fare`

In [7]:
fare_categories = pd.qcut(data.fare, 2)
fare_categories

0       (-0.001, 14.454]
1      (14.454, 512.329]
2       (-0.001, 14.454]
3      (14.454, 512.329]
4       (-0.001, 14.454]
             ...        
886     (-0.001, 14.454]
887    (14.454, 512.329]
888    (14.454, 512.329]
889    (14.454, 512.329]
890     (-0.001, 14.454]
Name: fare, Length: 891, dtype: category
Categories (2, interval[float64]): [(-0.001, 14.454] < (14.454, 512.329]]

In [8]:
data.pivot_table('survived', ['sex', age_categories], [fare_categories, 'class'], fill_value = "Sin Datos")

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]         Sin Datos  1.000000  0.714286          0.909091   
       (18, 80]        Sin Datos  0.880000  0.444444          0.972973   
male   (0, 18]         Sin Datos  0.000000  0.260870          0.800000   
       (18, 80]                0  0.098039  0.125000          0.391304   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 80]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 80]  0.030303  0.192308

Podemos especificar distintas funciones de agregación para distintas columnas del DataFrame original

In [9]:
data.pivot_table(index='sex', columns='class',
                    aggfunc={'survived':sum, 'fare':'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

Para calcular totales por grupo usamos `margins` y `margins_name`:

In [10]:
data.pivot_table('survived', index='sex', columns='class', margins=True, margins_name= "totales")

class,First,Second,Third,totales
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
totales,0.629630,0.472826,0.242363,0.383838


---

#### Referencias

Python for Data Analysis. Wes McKinney. Cap 10

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html